## 1. Kensu initialisation

To use the library, you need to add the kensu jar to the spark client

Initialize the client with the `KensuProvider` object with its `Context`:
- `process_name`:  the application name
- `project_names`: where the application is running 
- `environment`: ... well

In [ ]:
%%init_spark
launcher.jars = ["libs/kensu-dam-spark-collector-0.17.2_spark-3.0.1.jar"]
launcher.conf.set("spark.sql.shuffle.partitions", "1")

In [ ]:
val app = 2
val week = 1

In [ ]:
val properties = {
    import scala.collection.JavaConverters._
    val p = new java.util.Properties()
    p.load(new java.io.FileInputStream(s"./conf/application${app}-week${week}.properties"))
    scala.collection.mutable.Map.empty[String, Any]++=p.asScala.mapValues(v => scala.util.Try(v.toBoolean).orElse(scala.util.Try(v.toInt)).orElse(scala.util.Try(v.toLong)).getOrElse(v))
}
val users = List(properties("dam.activity.user"))
properties ++= Map("dam.activity.code.maintainers" -> users,"dam.activity.user" -> users(0),"dam.activity.projects" -> properties("dam.activity.projects").toString.split(","))
io.kensu.third.integration.TimeUtils.setMockedTime(properties("mocked.timestamp").asInstanceOf[Long], false)
val w = io.kensu.dam.lineage.spark.lineage.Implicits.SparkSessionDAMWrapper(spark)
w.track(Some(properties("dam.ingestion.url").toString), None)(properties.toList:_*)

 ## 2. Execution of the pipeline - App 2 : Total
 
 This application reads the parquet file created by App1 and computes a KPI : `Total`



In [ ]:
val df = spark.read.option("inferSchema","true").option("header","true").load(s"./data_${properties("dam.activity.user")}/week${week}/joined_data")

In [ ]:
df.show()

In [ ]:
df.createOrReplaceTempView("df")

In [ ]:
val Total_DF = spark.sql("SELECT id,price,quantity,round(price * quantity,2) AS total FROM df")

In [ ]:
Total_DF.show()

In [ ]:
Total_DF.write.mode("overwrite").save(s"./data_${properties("dam.activity.user")}/week${week}/data_total")